In [1]:
from torchvision import transforms
from PIL import Image
from torchvision import transforms
from PIL import Image

# Heavy augmentation for all non-Nepal countries
heavy_aug = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(12),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.5),
    transforms.RandomApply([transforms.RandomPerspective(distortion_scale=0.5)], p=0.5),
    transforms.ToTensor()
])

# Light/basic augmentation for Nepal only
light_aug = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

country_map = {
    "nepali": 0,
    "indian": 1,
    "bagladesh": 2,
    "pakistan": 3,
    "USA": 4,
    "euro": 5
}

denomination_map = {
    "5": 0,
    "10": 1,
    "20": 2,
    "50": 3,
    "100": 4,
    "500": 5,
    "1000": 6,
    "2000": 7,
    "5000": 8,
    "2":9,
    "1":10,
    "200":11
}



In [2]:
import os
from torch.utils.data import Dataset

class CurrencyDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        for country in os.listdir(root_dir):
            c_path = os.path.join(root_dir, country)
            for denom in os.listdir(c_path):
                d_path = os.path.join(c_path, denom)
                for img in os.listdir(d_path):
                    self.samples.append((
                        os.path.join(d_path, img),
                        country,
                        denom
                    ))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, country, denom = self.samples[idx]
        img = Image.open(img_path).convert("RGB")

        # Heavy augmentation for all except Nepal
        if country != "nepali":
            img = heavy_aug(img)
        else:
            img = light_aug(img)

        c_label = country_map[country]
        d_label = denomination_map[denom]
        return img, c_label, d_label



In [3]:
from torch.utils.data import DataLoader, random_split

dataset = CurrencyDataset(root_dir="/kaggle/input/paisaa/paisa(resized)/train")
train_size = int(0.8 * len(dataset))
val_size   = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [4]:
print("Number of images:", len(train_dataset))
print("Number of batches per epoch:", len(train_loader))


Number of images: 31154
Number of batches per epoch: 974


In [5]:
# import torch
# import torch.nn as nn
# import torchvision.models as models

# device = "cuda" if torch.cuda.is_available() else "cpu"

# num_countries = len(country_map)    # e.g., 6
# num_denoms    = len(denomination_map)  # e.g., 9

# class MultiTaskModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         backbone = models.resnet18(pretrained=True)
#         self.features = nn.Sequential(*list(backbone.children())[:-1])  # remove final fc
#         self.country_head = nn.Linear(backbone.fc.in_features, num_countries)
#         self.denom_head   = nn.Linear(backbone.fc.in_features, num_denoms)

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(x.size(0), -1)  # flatten
#         country_out = self.country_head(x)
#         denom_out   = self.denom_head(x)
#         return country_out, denom_out

# model = MultiTaskModel().to(device)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleMultiTaskCNN(nn.Module):
    def __init__(self, num_countries, num_denoms):
        super(SimpleMultiTaskCNN, self).__init__()
        
        # --- Shared CNN Backbone ---
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [B,32,224,224]
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B,32,112,112]
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B,64,56,56]
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                             # [B,128,28,28]
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))                 # [B,256,1,1]
        )

        # Flattened feature size = 256
        self.country_head = nn.Linear(256, num_countries)
        self.denom_head   = nn.Linear(256, num_denoms)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)        # flatten [B,256]
        country_out = self.country_head(x)
        denom_out   = self.denom_head(x)
        return country_out, denom_out




In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ImprovedMultiTaskCNN(nn.Module):
    def __init__(self, num_countries, num_denoms):
        super(ImprovedMultiTaskCNN, self).__init__()
        
        # --- Shared CNN Backbone ---
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),   # [B,64,224,224]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # extra conv for richer features
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),                              # [B,64,112,112]

            # Block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),                              # [B,128,56,56]

            # Block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),                              # [B,256,28,28]

            # Block 4
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))                  # [B,512,1,1]
        )

        # --- Task-specific Heads ---
        self.country_head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_countries)
        )

        self.denom_head = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_denoms)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # flatten [B,512]
        country_out = self.country_head(x)
        denom_out   = self.denom_head(x)
        return country_out, denom_out


In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_countries = len(country_map)    # e.g., 6
num_denoms    = len(denomination_map)  # e.g., 9
model = ImprovedMultiTaskCNN(num_countries, num_denoms).to(device)

In [9]:
import torch.nn.functional as F
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-4)

def compute_loss(country_pred, denom_pred, country_label, denom_label):
    loss_country = F.cross_entropy(country_pred, country_label)
    loss_denom   = F.cross_entropy(denom_pred, denom_label)
    return loss_country + loss_denom  # equal weighting


In [10]:
from tqdm import tqdm
import torch

num_epochs = 70        # more epochs
initial_lr = 1e-3        # starting learning rate

optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
# Cosine schedule or StepLR will give large steps early, smaller later
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
# ↓ halves LR every 10 epochs (you can tweak)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_country_train = 0
    correct_denom_train   = 0
    total_train = 0

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    for imgs, country_labels, denom_labels in tqdm(train_loader, desc="Training", leave=False):
        imgs = imgs.to(device)
        country_labels = country_labels.to(device)
        denom_labels   = denom_labels.to(device)

        optimizer.zero_grad()
        c_pred, d_pred = model(imgs)
        loss = compute_loss(c_pred, d_pred, country_labels, denom_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # ---- Training accuracy ----
        _, c_pred_labels = torch.max(c_pred, 1)
        _, d_pred_labels = torch.max(d_pred, 1)
        total_train += imgs.size(0)
        correct_country_train += (c_pred_labels == country_labels).sum().item()
        correct_denom_train   += (d_pred_labels == denom_labels).sum().item()

    scheduler.step()  # decrease learning rate
    train_loss = running_loss / len(train_loader)
    train_country_acc = correct_country_train / total_train
    train_denom_acc   = correct_denom_train / total_train

    # ---------------- Validation ---------------- #
    model.eval()
    correct_country = 0
    correct_denom   = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for imgs, country_labels, denom_labels in tqdm(val_loader, desc="Validating", leave=False):
            imgs = imgs.to(device)
            country_labels = country_labels.to(device)
            denom_labels   = denom_labels.to(device)

            c_pred, d_pred = model(imgs)
            loss = compute_loss(c_pred, d_pred, country_labels, denom_labels)
            val_loss += loss.item()

            _, c_pred_labels = torch.max(c_pred, 1)
            _, d_pred_labels = torch.max(d_pred, 1)

            total += imgs.size(0)
            correct_country += (c_pred_labels == country_labels).sum().item()
            correct_denom   += (d_pred_labels == denom_labels).sum().item()

    val_loss /= len(val_loader)
    val_country_acc = correct_country / total
    val_denom_acc   = correct_denom / total

    # ---- Epoch summary ----
    print(f"  ➜ Train Loss: {train_loss:.4f} "
          f"| Country Acc: {train_country_acc:.4f} "
          f"| Denom Acc: {train_denom_acc:.4f}")
    print(f"  ➜ Val   Loss: {val_loss:.4f} "
          f"| Country Acc: {val_country_acc:.4f} "
          f"| Denom Acc: {val_denom_acc:.4f}")
    print(f"  ➜ Learning Rate: {scheduler.get_last_lr()[0]:.6f}")



Epoch [1/70]


  ➜ Train Loss: 2.2920 | Country Acc: 0.7618 | Denom Acc: 0.4301
  ➜ Val   Loss: 2.5604 | Country Acc: 0.6831 | Denom Acc: 0.4223
  ➜ Learning Rate: 0.001000

Epoch [2/70]


  ➜ Train Loss: 1.5839 | Country Acc: 0.8303 | Denom Acc: 0.6039
  ➜ Val   Loss: 1.4699 | Country Acc: 0.8485 | Denom Acc: 0.6220
  ➜ Learning Rate: 0.001000

Epoch [3/70]


  ➜ Train Loss: 1.2802 | Country Acc: 0.8625 | Denom Acc: 0.6696
  ➜ Val   Loss: 1.6783 | Country Acc: 0.8221 | Denom Acc: 0.5536
  ➜ Learning Rate: 0.001000

Epoch [4/70]


  ➜ Train Loss: 1.1248 | Country Acc: 0.8878 | Denom Acc: 0.7013
  ➜ Val   Loss: 1.1246 | Country Acc: 0.8875 | Denom Acc: 0.7030
  ➜ Learning Rate: 0.001000

Epoch [5/70]


...

  ➜ Train Loss: 0.0878 | Country Acc: 0.9971 | Denom Acc: 0.9746
  ➜ Val   Loss: 0.0780 | Country Acc: 0.9972 | Denom Acc: 0.9791
  ➜ Learning Rate: 0.000016

Epoch [67/70]


  ➜ Train Loss: 0.0859 | Country Acc: 0.9972 | Denom Acc: 0.9749
  ➜ Val   Loss: 0.0872 | Country Acc: 0.9965 | Denom Acc: 0.9770
  ➜ Learning Rate: 0.000016

Epoch [68/70]


  ➜ Train Loss: 0.0857 | Country Acc: 0.9975 | Denom Acc: 0.9749
  ➜ Val   Loss: 0.0865 | Country Acc: 0.9977 | Denom Acc: 0.9757
  ➜ Learning Rate: 0.000016

Epoch [69/70]


  ➜ Train Loss: 0.0876 | Country Acc: 0.9974 | Denom Acc: 0.9745
  ➜ Val   Loss: 0.0862 | Country Acc: 0.9981 | Denom Acc: 0.9750
  ➜ Learning Rate: 0.000016

Epoch [70/70]


  ➜ Train Loss: 0.0878 | Country Acc: 0.9975 | Denom Acc: 0.9756
  ➜ Val   Loss: 0.0850 | Country Acc: 0.9978 | Denom Acc: 0.9784
  ➜ Learning Rate: 0.000008


In [12]:
import torch

# Save the model state dict (weights only)
torch.save(model.state_dict(), "currency_model_weights.pth")


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
# from model import ImprovedMultiTaskCNN   # <-- if saved in model.py

# ----------------- CONFIG -----------------
MODEL_PATH     = "/kaggle/input/improved-97-per/pytorch/default/1/currency_model_weights (3)(97).pth"     # Trained weights
IMG_PATH       = "/kaggle/input/paisaa/paisa(resized)/train/bagladesh/10/IMG-20230826-WA0001.jpg"      # Test image
NUM_COUNTRIES  = 6             # update with your dataset
NUM_DENOMS     = 12             # update with your dataset
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"

# Label mappings (example)
country_labels = ["Nepal", "India",  "Bangladesh", "Bhutan","Pakistan",
                  "usa", "euro"]
denom_labels   = ["5", "10", "20", "50", "100", "500", "1000", "2000","5000","2","1","200"]

# ----------------- LOAD MODEL -----------------
model = ImprovedMultiTaskCNN(num_countries=NUM_COUNTRIES,
                             num_denoms=NUM_DENOMS)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# ----------------- PREPROCESS IMAGE -----------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

img = Image.open(IMG_PATH).convert("RGB")
img_tensor = transform(img).unsqueeze(0).to(DEVICE)   # [1,3,224,224]

# ----------------- INFERENCE -----------------
with torch.no_grad():
    country_logits, denom_logits = model(img_tensor)
    country_pred = country_logits.argmax(dim=1).item()
    denom_pred   = denom_logits.argmax(dim=1).item()

print(f"Predicted Country: {country_labels[country_pred]}")
print(f"Predicted Denomination: {denom_labels[denom_pred]}")
